In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D,MaxPooling1D,GlobalMaxPooling1D,Dense,LSTM
from keras.models import Input,Model
from keras.callbacks import ModelCheckpoint

In [4]:
df = pd.read_csv('train.csv')

In [5]:
df.head(25)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [6]:
df1=pd.read_csv('test.csv')
df1.head(4)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."


In [7]:
sentences = df['comment_text']
sentences.isnull().sum()

0

In [8]:
sentences

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [9]:
tokenizer = Tokenizer(num_words=20000) #to keep max token id 20000
tokenizer.fit_on_texts(sentences)
sequence = tokenizer.texts_to_sequences(sentences)

In [10]:
sequence[0]

[688,
 75,
 1,
 126,
 130,
 177,
 29,
 672,
 4511,
 12052,
 1116,
 86,
 331,
 51,
 2278,
 11448,
 50,
 6864,
 15,
 60,
 2756,
 148,
 7,
 2937,
 34,
 117,
 1221,
 15190,
 2825,
 4,
 45,
 59,
 244,
 1,
 365,
 31,
 1,
 38,
 27,
 143,
 73,
 3462,
 89,
 3085,
 4583,
 2273,
 985]

In [11]:
classes=["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
target = df[classes].values

In [12]:
target

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [13]:
type(target)

numpy.ndarray

In [14]:
word2idx = tokenizer.word_index #inbuilt parameter

In [15]:
word2idx #dictionary

{'the': 1,
 'to': 2,
 'of': 3,
 'and': 4,
 'a': 5,
 'you': 6,
 'i': 7,
 'is': 8,
 'that': 9,
 'in': 10,
 'it': 11,
 'for': 12,
 'this': 13,
 'not': 14,
 'on': 15,
 'be': 16,
 'as': 17,
 'have': 18,
 'are': 19,
 'your': 20,
 'with': 21,
 'if': 22,
 'article': 23,
 'was': 24,
 'or': 25,
 'but': 26,
 'page': 27,
 'wikipedia': 28,
 'my': 29,
 'an': 30,
 'from': 31,
 'by': 32,
 'do': 33,
 'at': 34,
 'me': 35,
 'about': 36,
 'so': 37,
 'talk': 38,
 'what': 39,
 'can': 40,
 'there': 41,
 'all': 42,
 'has': 43,
 'will': 44,
 'please': 45,
 'no': 46,
 'would': 47,
 'one': 48,
 'like': 49,
 'just': 50,
 'they': 51,
 'he': 52,
 'which': 53,
 'any': 54,
 'been': 55,
 'should': 56,
 'more': 57,
 'we': 58,
 "don't": 59,
 'some': 60,
 'other': 61,
 'who': 62,
 'here': 63,
 'see': 64,
 'also': 65,
 'his': 66,
 'think': 67,
 'because': 68,
 'know': 69,
 'how': 70,
 'edit': 71,
 'am': 72,
 "i'm": 73,
 'people': 74,
 'why': 75,
 'up': 76,
 'only': 77,
 "it's": 78,
 'out': 79,
 'articles': 80,
 'use': 81,

In [16]:
sum1=0
for item in sequence:
    sum1+=len(item)
sum1

10438283

In [17]:
avg_seq_len = sum1/(len(sequence))

In [18]:
avg_seq_len

65.41466181198338

In [19]:
max_len=100 #to pad.

In [20]:
data = pad_sequences(sequence,maxlen=max_len)

In [21]:
data[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,   688,
          75,     1,   126,   130,   177,    29,   672,  4511, 12052,
        1116,    86,   331,    51,  2278, 11448,    50,  6864,    15,
          60,  2756,   148,     7,  2937,    34,   117,  1221, 15190,
        2825,     4,    45,    59,   244,     1,   365,    31,     1,
          38,    27,   143,    73,  3462,    89,  3085,  4583,  2273,
         985])

In [22]:
#Word Vectors Loading
word2vec = {}
with open('C:/Users/Asus/Desktop/ML PROJECTS/Glove Vectors/glove.6B.100d.txt',"r",encoding="utf-8") as f:
    for lines in f:
        value = lines.split()
        word=value[0]
        vector=np.asarray(value[1:],dtype="float32")
        word2vec[word]=vector

        

        
   


In [23]:
word2vec['the']

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

In [24]:
max_vocab_size=20000
numwords=min(max_vocab_size,len(word2idx)+1) # as keras tokenizes from 1
emb_matrix=np.zeros((numwords,100)) #100 dim vectors
for word,index in word2idx.items():
    if index<max_vocab_size:
        vector = word2vec.get(word) #don't use [''] to prevent eroor,.get returns null if word not found.
        if vector is not None:
            emb_matrix[index]=vector

In [25]:
emb_matrix[1]

array([-0.038194  , -0.24487001,  0.72812003, -0.39961001,  0.083172  ,
        0.043953  , -0.39140999,  0.3344    , -0.57545   ,  0.087459  ,
        0.28786999, -0.06731   ,  0.30906001, -0.26383999, -0.13231   ,
       -0.20757   ,  0.33395001, -0.33848   , -0.31742999, -0.48335999,
        0.1464    , -0.37303999,  0.34577   ,  0.052041  ,  0.44946   ,
       -0.46970999,  0.02628   , -0.54154998, -0.15518001, -0.14106999,
       -0.039722  ,  0.28277001,  0.14393   ,  0.23464   , -0.31020999,
        0.086173  ,  0.20397   ,  0.52623999,  0.17163999, -0.082378  ,
       -0.71787   , -0.41531   ,  0.20334999, -0.12763   ,  0.41367   ,
        0.55186999,  0.57907999, -0.33476999, -0.36559001, -0.54856998,
       -0.062892  ,  0.26583999,  0.30204999,  0.99774998, -0.80480999,
       -3.0243001 ,  0.01254   , -0.36941999,  2.21670008,  0.72201002,
       -0.24978   ,  0.92136002,  0.034514  ,  0.46744999,  1.10790002,
       -0.19358   , -0.074575  ,  0.23353   , -0.052062  , -0.22

In [26]:
#embedding layer
embedding_layer = Embedding(numwords,100,weights=[emb_matrix],input_length=max_len,trainable=False)

In [31]:
inputs = Input(shape=(max_len,))
x=embedding_layer(inputs)
for i in range(3):
    x=Conv1D(128,3,activation='relu')(x)
    if(i<2):
        x = MaxPooling1D(3)(x)
    if(i==2):
        x=GlobalMaxPooling1D()(x)
output=Dense(6,activation='sigmoid')(x)
model = Model(inputs,output)
model.summary()




Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 100)          2000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 128)           38528     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 32, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 30, 128)           49280     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 10, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 8, 128)            4928

In [43]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [42]:
model_checkpoints =ModelCheckpoint(
    filepath='best_weights.h5',
    save_weights_only=True,
    monitor='val_acc',
    verbose=1,
    mode='max',
    save_best_only=True)

In [45]:
r=model.fit(data,target,batch_size=64,epochs=10,validation_split=0.2,callbacks=[model_checkpoints])

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 30s 235us/step - loss: 0.0796 - accuracy: 0.9744 - val_loss: 0.0683 - val_accuracy: 0.9770
Epoch 2/10
  1408/127656 [..............................] - ETA: 15s - loss: 0.0767 - accuracy: 0.9734

C:\Users\Asus\Anaconda3\envs\gputest\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


127656/127656 [==============================] - 16s 125us/step - loss: 0.0650 - accuracy: 0.9779 - val_loss: 0.0657 - val_accuracy: 0.9778
Epoch 3/10
127656/127656 [==============================] - 16s 126us/step - loss: 0.0604 - accuracy: 0.9791 - val_loss: 0.0679 - val_accuracy: 0.9766
Epoch 4/10
127656/127656 [==============================] - 16s 128us/step - loss: 0.0560 - accuracy: 0.9803 - val_loss: 0.0664 - val_accuracy: 0.9773
Epoch 5/10
127656/127656 [==============================] - 16s 127us/step - loss: 0.0520 - accuracy: 0.9815 - val_loss: 0.0712 - val_accuracy: 0.9774
Epoch 6/10
127656/127656 [==============================] - 17s 134us/step - loss: 0.0479 - accuracy: 0.9828 - val_loss: 0.0704 - val_accuracy: 0.9760
Epoch 7/10
127656/127656 [==============================] - 16s 129us/step - loss: 0.0444 - accuracy: 0.9840 - val_loss: 0.0725 - val_accuracy: 0.9766
Epoch 8/10
127656/127656 [==============================] - 17s 136us/step - loss: 0.0406 - accuracy: 0.9

In [46]:
model.save_weights('best_weights.h5')

In [32]:
test_df=pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [33]:
test_df

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [34]:
test_inputs = test_df['comment_text']

In [35]:
test_Seq = tokenizer.texts_to_sequences(test_inputs)

In [36]:
datatest = pad_sequences(test_Seq,maxlen=100)

In [68]:
results=model.predict(data)

In [89]:
results.shape

(153164, 6)

In [108]:
result2=np.zeros(results.shape,dtype=int)

In [109]:
result2=np.zeros(results.shape,dtype=int)
for i in range(results.shape[0]):
    for j in range(results.shape[1]):
        if(results[i][j]>=0.5):
            result2[i][j]=int(1.0)
        else:
            result2[i][j]=int(0)

array([[1., 1., 1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0.]])

In [41]:
df3=pd.read_csv('test_labels.csv')
df3

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,-1,-1,-1,-1,-1,-1


In [110]:
df3[classes]=result2

In [111]:
df3.head()


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1,1,1,0,1,1
1,0000247867823ef7,0,0,0,0,0,0
2,00013b17ad220c46,0,0,0,0,0,0
3,00017563c3f7919a,0,0,0,0,0,0
4,00017695ad8997eb,0,0,0,0,0,0


In [107]:
test_df['comment_text'][0]

"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"

In [121]:
df3.to_csv('finaltoxicpredictions.csv',index=False)

In [122]:
sample

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.5,0.5,0.5,0.5,0.5,0.5
153160,fffd7a9a6eb32c16,0.5,0.5,0.5,0.5,0.5,0.5
153161,fffda9e8d6fafa9e,0.5,0.5,0.5,0.5,0.5,0.5
153162,fffe8f1340a79fc2,0.5,0.5,0.5,0.5,0.5,0.5


In [218]:
comment=["Yo don;t mess with me nigger"]

In [219]:
comseq=tokenizer.texts_to_sequences(comment)
query = pad_sequences(comseq,maxlen=100)
ans=model.predict(query)

In [220]:
ans1=np.array(ans,dtype='float16')

In [221]:
ans1

array([[0.7134  , 0.0676  , 0.4797  , 0.010925, 0.4583  , 0.06226 ]],
      dtype=float16)

In [27]:
#Using LSTM Model
inputs = Input(shape=(100,))
x=embedding_layer(inputs)
x=LSTM(15,return_sequences=True)(x)
x=GlobalMaxPooling1D()(x)
op=Dense(6,activation='sigmoid')(x)
model1 = Model(inputs,op)



In [28]:
model1.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
model_checkpoints =ModelCheckpoint(
    filepath='LSTM_best_weights.h5',
    save_weights_only=True,
    monitor='val_acc',
    verbose=1,
    mode='max',
    save_best_only=True)

In [30]:
model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 100)          2000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 15)           6960      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 15)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 96        
Total params: 2,007,056
Trainable params: 7,056
Non-trainable params: 2,000,000
_________________________________________________________________


In [31]:
r=model1.fit(data,target,batch_size=64,epochs=4,validation_split=0.2,callbacks=[model_checkpoints])

Train on 127656 samples, validate on 31915 samples
Epoch 1/4
127656/127656 [==============================] - 409s 3ms/step - loss: 0.1178 - accuracy: 0.9688 - val_loss: 0.0630 - val_accuracy: 0.9787
Epoch 2/4
    64/127656 [..............................] - ETA: 5:14 - loss: 0.0930 - accuracy: 0.9714

C:\Users\Asus\Anaconda3\envs\gputest\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


127656/127656 [==============================] - 398s 3ms/step - loss: 0.0588 - accuracy: 0.9798 - val_loss: 0.0581 - val_accuracy: 0.9799
Epoch 3/4
127656/127656 [==============================] - 407s 3ms/step - loss: 0.0550 - accuracy: 0.9808 - val_loss: 0.0554 - val_accuracy: 0.9806
Epoch 4/4
127656/127656 [==============================] - 379s 3ms/step - loss: 0.0527 - accuracy: 0.9814 - val_loss: 0.0532 - val_accuracy: 0.9808


In [37]:
results=model1.predict(data)

In [38]:
result2=np.zeros(results.shape,dtype=int)
for i in range(results.shape[0]):
    for j in range(results.shape[1]):
        if(results[i][j]>=0.5):
            result2[i][j]=int(1.0)
        else:
            result2[i][j]=int(0)

In [45]:
#result2.shape

(159571, 6)

153164